The following notebook will be split into 4 parts:
 - [Part 1](#Part1_UCID): Setup
 - [Part 2](#Part2_UCID): Exploratory Data Analysis
 - [Part 3](#Part3_UCID): Clustering
 - [Part 4](#Part4_UCID): Forecasting
 
**Part 1:** Initial setup (importing relevant packages, setting up global hyperparameters, importing/cleaning our data set).

**Part 2:** Exploratory data analysis that serves to act as the feature selection step by determining which features are ir/relevant.

**Part 3:** Clustering days based on a similarity metric.

**Part 4:** Forecasting on a per-cluster basis.

# Part 1: Setup <a id="Part1_UCID"> </a>

## 1.1: Initial setup

**Step 1:** Import the relevant packages and set Seaborn/Matplotlib hyperparameters.

In [ ]:
import holidays
import os

import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from scipy import stats
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from tqdm import notebook

plt.style.use("fivethirtyeight")
sns.set(style="whitegrid", palette="muted")

plt.rcParams['agg.path.chunksize'] = 10000
plt.rcParams["axes.labelsize"] = 26
plt.rcParams["axes.titlesize"] = 26
plt.rcParams["figure.figsize"] = 16, 10
plt.rcParams["figure.dpi"] = 300
plt.rcParams["xtick.labelsize"] = 22
plt.rcParams["ytick.labelsize"] = 22

np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.2f}'.format

%config Completer.use_jedi = False

**Step 2:** Define the location of our data as well as the relevant columns that we would like to include.

In [ ]:
cols_UCID = [
    "Date",
    "Time",
    "Global_active_power",
    "Global_reactive_power",
    "Voltage",
    "Global_intensity",
    "Sub_metering_1",
    "Sub_metering_2",
    "Sub_metering_3",
]

data_directory_UCID = os.path.join("Data", "UCI")
data_directory_Solcast = os.path.join("Data", "Solcast_UCID")

house = "household_power_consumption.txt"
solcast_15 = "Solcast_UCID_15.csv"

file_destination_UCID = os.path.join(data_directory_UCID, house)
file_destination_Solcast = os.path.join(data_directory_Solcast, solcast_15)

**Step 3:** Read in the data and save it to a dataframe.

In [ ]:
df_UCID = pd.read_csv(file_destination_UCID, parse_dates=True,  delimiter = ";", usecols=cols_UCID)

df_UCID["DT"] = df_UCID["Date"].str.cat(df_UCID["Time"], sep=" ")
df_UCID["DT"] = pd.to_datetime(df_UCID["DT"], format="%d/%m/%Y %H:%M:%S")

df_UCID = df_UCID.reset_index()
df_UCID = df_UCID.set_index("DT")
df_UCID.index = pd.to_datetime(df_UCID.index)

cols_NA = [
    "index",
    "Date",
    "Time",
]

df_UCID.drop(cols_NA, axis=1, inplace=True)

cols = df_UCID.columns
df_UCID[cols] = df_UCID[cols].apply(pd.to_numeric, errors="coerce")

cols_UCID.remove("Date")
cols_UCID.remove("Time")

In [ ]:
df_Solcast = pd.read_csv(file_destination_Solcast, index_col=0, parse_dates=True)

df_Solcast.index = df_Solcast.index.rename("Time")
df_Solcast.index = pd.to_datetime(df_Solcast.index).tz_localize(None)

## 1.2: Scale the data in the dataframe(s)

**Step 1.1:** Scale the data in a range between 0 and 1 (optional).

In [ ]:
# minmax_UCID = MinMaxScaler()
# minmax_Solcast = MinMaxScaler()

# df_UCID[cols_UCID] = minmax_UCID.fit_transform(df_REFIT[cols_UCID])
# df_Solcast[cols_Solcast] = minmax_Solcast.fit_transform(df_Solcast[cols_Solcast])

**Step 1.2:** Standardize the data by removing the mean and scaling to unit variance (optional).

In [ ]:
standardscale_UCID = StandardScaler()
standardscale_Solcast = StandardScaler()

df_UCID[cols_UCID] = standardscale_UCID.fit_transform(df_UCID[cols_UCID])
df_Solcast[cols_Solcast] = standardscale_Solcast.fit_transform(df_Solcast[cols_Solcast])

## 1.3: Merge the dataframes

**Step 1:** Create a copy of our REFIT dataframe that is resampled into a resolution of 15 minutes and drop any days that contain an incomplete number of values.

In [ ]:
df_UCID_resampled = df_UCID.resample("15min").mean()
df_UCID_resampled = df_UCID_resampled.dropna()

mask = df_UCID_resampled.groupby(df_UCID_resampled.index.date).size()
mask = mask[mask < 96].index.to_list()

df_UCID_resampled = df_UCID_resampled[~df_UCID_resampled.index.floor("D").isin(mask)]

**Step 2:** Create a third dataframe that is the result of merging the Solcast dataframe with the REFIT dataframe.

In [ ]:
df_Merged = pd.merge(left=df_Solcast, left_on=df_Solcast.index, right=df_UCID_resampled, right_on=df_UCID_resampled.index)

cols_Merged = [
    "PeriodStart",
    "Period",
    "Global_reactive_power",
    "Voltage",
    "Global_intensity",
    "Sub_metering_1",
    "Sub_metering_2",
    "Sub_metering_3",
]

df_Merged.drop(cols_Merged, axis=1, inplace=True)
df_Merged.rename(columns={"key_0": "Time"}, inplace=True)
df_Merged = df_Merged.set_index("Time")
df_Merged.index = pd.to_datetime(df_Merged.index)
df_Merged.head()

## 1.4: Append temporal features to our merged dataframe

**Step 1:** Append public holidays to our merged dataframe.

In [ ]:
France_holidays = holidays.France()
df_Merged.insert(0, "Holiday", [1 if str(val).split()[0] in France_holidays else 0 for val in df_Merged.index.date])
df_Merged["Holiday"] = df_Merged["Holiday"].astype("category")

**Step 2:** Define day of the year ranges for each of the seasons.

In [ ]:
spring = range(60, 152)
summer = range(152, 244)
fall = range(244, 336)

def season(doy):
    if doy in spring:
        return "0"
    if doy in summer:
        return "1"
    if doy in fall:
        return "2"
    else:
        return "3"

**Step 3:** Append temporal data to our merged dataframe.

In [ ]:
df_Merged.insert(0, "Year", df_Merged.index.year)
df_Merged.insert(1, "Month", df_Merged.index.month)
df_Merged.insert(3, "Day", df_Merged.index.day)
df_Merged.insert(4, "Hour", df_Merged.index.hour)
df_Merged.insert(5, "Minute", df_Merged.index.minute)
df_Merged.insert(6, "Weekday", df_Merged.index.weekday)
df_Merged.insert(7, "Season", df_Merged.index.dayofyear.map(season))

# Part 2: Exploratory Data Analysis <a id="Part2_UCID"> </a>

## 2.1: Visualize the data

### 2.1.1: Line plot

**Step 1:** Create a copy of our REFIT dataframe and reformat the index into a string so as to make sure that Matplotlib does not automatically interpolate missing values.

In [ ]:
df_UCID_resampled_c = df_UCID_resampled.copy()
df_UCID_resampled_c.index = df_UCID_resampled_c.index.strftime("%d-%m-%y %H:%M:%S")

**Step 2:** Drop all columns barre the `Aggregate` column.

In [ ]:
cols_UCID_c = cols_UCID.copy()
cols_UCID_c.remove("Global_active_power")
df_UCID_resampled_c.drop(cols_UCID_c, axis=1, inplace=True)

**Step 3:** Plot the aggregate power consumption over time.

In [ ]:
fig, ax = plt.subplots()
df_UCID_resampled_c["Global_active_power"].plot(ax=ax)

ax.set_xlabel("")
ax.set_ylabel("Aggregate Power Consumption (Watts)")
ax.set_xlim(left=-5, right=len(df_UCID_resampled_c) + 5)
ax.set_ylim(bottom=df_UCID_resampled_c["Global_active_power"].min())

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

### 2.1.2: Count plot

**Step 1:** Visualize the number of samples per day of the week over the entire data set.

In [ ]:
df_Merged_c = df_Merged.copy()
df_Merged_c.insert(0, "Weekday_name", df_Merged.index.day_name())

ax = sns.countplot(
    x=df_Merged_c["Weekday_name"],
    data=df_Merged_c,
    palette="icefire",
    order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
)

plt.xlabel("Weekday")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

**Step 2:** Visualize the number of samples per month over the entire data set.

In [ ]:
df_Merged_c = df_Merged.copy()
df_Merged_c.insert(0, "Month_name", df_Merged.index.month_name())

ax = sns.countplot(
    x=df_Merged_c["Month_name"],
    data=df_Merged_c,
    palette="icefire",
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

### 2.1.3: Stationarity plot

**Step 1:** Test for stationarity and plot the result using the pre-defined `test_stationarity` function.

In [ ]:
test_stationarity(df_UCID_resampled_c, 0.05, "Global_active_power", "Aggregate Power Consumption (Watts)")

### 2.1.4: Box and whisker plots/outliers

**Step 1:** Box and whiskers plot for the aggregate as well as each of the IAM readings grouped by the months of the year.

In [ ]:
fig, axs = plt.subplots(len(cols_UCID), 1, figsize=(16, 10 * (len(cols_UCID))), sharex=True)

for col, ax in zip([*cols_UCID], axs):
    sns.boxplot(
        data=df_UCID,
        x=df_UCID.index.month_name(),
        y=col,
        ax=ax,
        order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    )
    ax.set_title(col)
    ax.xaxis.set_label_text("")

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.tight_layout()
plt.show()

**Step 2:** We remove outliers that are 3 standard deviations away from the mean and repeat **step 1**.

In [ ]:
df_out = df_UCID[(np.abs(stats.zscore(df_UCID["Global_active_power"], nan_policy="omit")) < 3)]
print(f"Number of outliers: {(len(df_UCID) - len(df_out))}")

In [ ]:
len(df_UCID)

In [ ]:
fig, axs = plt.subplots(len(cols_UCID), 1, figsize=(16, 10 * (len(cols_UCID))), sharex=True)

for col, ax in zip([*cols_UCID], axs):
    sns.boxplot(
        data=df_out,
        x=df_out.index.month_name(),
        y=col,
        ax=ax,
        order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    )
    ax.xaxis.set_label_text("")

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.tight_layout()
plt.show()

**Step 2.1:** Purely for the global active power.

In [ ]:
ax = sns.boxplot(
    data=df_out,
    x=df_out.index.month_name(),
    y=df_out["Global_active_power"],
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Aggregate Power Consumption (Watts)")
plt.tight_layout()
plt.show()

**Step 3:** Box and whiskers plot for the aggregate as well as each of the IAM readings over the entirety of the data set.

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_out, ax=ax)

plt.setp(ax.get_xticklabels(), ha="right", rotation=45)
plt.tight_layout()
plt.show()

## 2.2: Time series decomposition

**Step 1:** Create a copy of our REFIT dataframe and reformat the index into a string so as to make sure that Matplotlib does not automatically interpolate missing values.

In [ ]:
df_UCID_resampled_c = df_UCID_resampled.copy()
df_UCID_resampled_c.index = df_UCID_resampled_c.index.strftime("%d-%m-%y %H:%M:%S")

**Step 2:** Drop all columns barre the `Aggregate` column.

In [ ]:
cols_UCID_c = cols_UCID.copy()
cols_UCID_c.remove("Global_active_power")
df_UCID_resampled_c.drop(cols_UCID_c, axis=1, inplace=True)

In [ ]:
df_UCID_resampled_c

**Step 3:** Define a period of 1 year (as an example).

In [ ]:
df_UCID_resampled_c = df_UCID_resampled_c.loc["01-01-07 00:00:00":"01-01-08 00:00:00"]

**Step 4:** Perform time series decomposition using LOESS.

In [ ]:
freq = (24 * 60) // 15
stl_decompose_result = STL(df_UCID_resampled_c, period=freq).fit()

**Step 5:** Separate/plot the obtained results.

In [ ]:
observed = stl_decompose_result.observed
trend = stl_decompose_result.trend.to_frame()
seasonal = stl_decompose_result.seasonal.to_frame()
noise = stl_decompose_result.resid.to_frame()

In [ ]:
fig, axs = plt.subplots(4, sharex=True)

observed.plot(ax=axs[0], title="Observed")
trend.plot(ax=axs[1], title="Trend")
seasonal.plot(ax=axs[2], title="Seasonal")
noise.plot(ax=axs[3], title="Noise")

for ax in axs:
    ax.set_xlabel("")
    ax.set_xlim(0, right=len(observed))
    ax.get_legend().remove()
    ax.title.set_size(22)

fig.text(0.03, 0.6, "Aggregate Power Consumption (Kilowatts)", fontsize="22", va="center", rotation="vertical")
plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

## 2.3: Causality and correlation

**Step 1:** Perform the Augmented Dicky-Fuller test to determine whether our time series is stationary.

In [ ]:
for name, column in df_Merged.iteritems():
    adfuller_test(column, name=column.name)
    print("\n")

**Step 2:** Check for autocorrelation.

In [ ]:
plot_acf(df_UCID_resampled["Global_active_power"], lags=192, zero=False)
plt.show()

**Step 3:** Check for partial autocorrelation.

In [ ]:
plot_pacf(df_UCID_resampled["Global_active_power"], lags=96, zero=False)
plt.show()

**Step 4:** Check for highly correlated features in the Solcast dataframe and drop them from our merged dataframe.

In [ ]:
highly_correlated = correlation(df_Solcast, 0.7, df_Merged.Global_active_power)
A = len(df_Merged.columns)
print(highly_correlated)
#df_Merged.drop(highly_correlated, axis=1, inplace=True)
B = len(df_Merged.columns) - len(highly_correlated)
print(A, B)

**Step 5:** Estimate mutual information of our independent variables on our target variable.

In [ ]:
X, y = (df_Merged.loc[:, df_Merged.columns != "Global_active_power"], df_Merged["Global_active_power"])
mutual_info = mutual_info_regression(X, y)

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info = mutual_info.sort_values(ascending=False)

In [ ]:
ax = sns.barplot(x=mutual_info.values, y=mutual_info.index, palette="icefire")
ax.set_xlabel("Mutual information")
ax.set_ylabel("Independent variable")
plt.tight_layout()
plt.show()

**Step 6:** Granger causality test.

In [ ]:
gm = grangers_causation_matrix(df_Merged, variables = df_Merged.columns) 

In [ ]:
gm2 = gm.transpose()["Global_active_power_y"].iloc[0:18].to_frame()
ax = sns.heatmap(gm2, mask=gm2 < 0.05, annot=True, fmt=".2f", square=True, cmap="rocket_r", vmin=0.0, vmax=1.0)
ax2 = sns.heatmap(gm2, mask=gm2 >= 0.05, annot=True, fmt=".2f", square=True, cmap="rocket_r", vmin=0.0, vmax=1.0, annot_kws={"weight": "bold"}, cbar=False)

plt.show()

In [ ]:
ax = sns.heatmap(gm.transpose(), annot=True, fmt=".2f", square=True, cmap="rocket_r")

In [ ]:
# gm2 = gm.where(np.tril(np.ones(gm.shape)).astype(bool))
# cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)

# sns.heatmap(gm2, center=0, annot=True, fmt=".2f", square=True, cmap=cmap)
# plt.show()

# Part 3: Clustering <a id="Part3_UCID"> </a>

# Part 4: Forecasting <a id="Part4_UCID"> </a>

# Appendix

## Miscellaneous functions

### 1) Augmented Dickey–Fuller test

In [ ]:
def adfuller_test(series, signif=0.05, name=""):
    r = adfuller(series, autolag="AIC")
    output = {"test_statistic": round(r[0], 4), "pvalue": round(r[1], 4), "n_lags": round(r[2], 4), "n_obs": r[3]}
    p_value = output["pvalue"]

    def adjust(val, length=6):
        return str(val).ljust(length)

    print(f'      Augmented Dickey-Fuller Test on "{name}"', "\n   ", "-" * 47)
    print(f" Null Hypothesis: Data has unit root. Non-Stationary.")
    print(f" Significance Level    = {signif}")
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key, val in r[4].items():
        print(f" Critical value {adjust(key)} = {round(val, 3)}")
    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

### 2) Augmented Dickey–Fuller test w/plot

In [ ]:
def test_stationarity(series, signif=0.05, name="", ylabel=""):
    def adjust(val, length=6):
        return str(val).ljust(length)

    rolmean = series.rolling(12).mean()
    rolstd = series.rolling(12).std()

    fig, ax = plt.subplots()

    series.plot(ax=ax, alpha=0.5)
    rolmean.plot(ax=ax, alpha=0.7)
    rolstd.plot(ax=ax, alpha=0.7)

    ax.set_xlabel("")
    ax.set_ylabel(ylabel)
    ax.set_xlim(left=0, right=len(series))
    plt.legend(loc="best")
    plt.title("Rolling Mean & Standard Deviation")
    plt.setp(ax.get_xticklabels(), ha="right", rotation=60)

    leg = plt.legend()
    leg.get_texts()[0].set_text(name)
    leg.get_texts()[1].set_text("Rolling Mean")
    leg.get_texts()[2].set_text("Rolling STD")

    plt.tight_layout()
    plt.show(block=False)

    adfuller_test(series, 0.05, name)

### 3) Granger Causality test

In [ ]:
def grangers_causation_matrix(data, variables, test="ssr_chi2test", maxlag=12, verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = ([round(test_result[i + 1][0][test][1], 2) for i in range(maxlag)])
            if verbose:
                print(f"Y = {r}, X = {c}, P Values = {p_values}")
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + "_x" for var in variables]
    df.index = [var + "_y" for var in variables]
    return df

### 4) Determine which highly correlated independent variables have a stronger correlation with our target variable.

In [ ]:
def correlation(df, threshold, target_variable):
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                rowname = corr_matrix.index[j]
                cor1 = abs(df[colname].corr(target_variable))
                cor2 = abs(df[rowname].corr(target_variable))
                if  cor1 > cor2:
                    col_corr.add(corr_matrix.index[j])
                else:
                    col_corr.add(corr_matrix.columns[i])
    return col_corr

### 5) Reshape correlation matrix

In [ ]:
def reshape_corr(df):
    df_corr = df.corr().stack().reset_index()
    df_corr.columns = ["Feature 1", "Feature 2", "Correlation"]
    mask_dups = (df_corr[["Feature 1", "Feature 2"]].apply(frozenset, axis=1).duplicated()) | (df_corr["Feature 1"] == df_corr["Feature 2"])
    df_corr = df_corr[~mask_dups]

    return df_corr